<a href="https://colab.research.google.com/github/ammarSherif/CIT690E-Deep-Learning-Labs/blob/main/Lab%205%3A%20pre-trained%20models%20%5BVGG16%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 5: Pre-trained models [VGG16]

CIT690E: Deep Learning <br>
Nile University<br>
Ammar Sherif<br>
Github Repo: [CIT690E-Deep-Learning-Labs](https://github.com/ammarSherif/CIT690E-Deep-Learning-Labs)

In [1]:
# ==============================================================================
# Some imports that we are using in our lab
# ==============================================================================
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms,models,datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch import optim
import cv2, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

In [2]:
# ==============================================================================
# Identify the device at the beginning
# ==============================================================================
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


## Dataset

In [3]:
# ==============================================================================
# We are using CIFAR-10 dataset
# ==============================================================================
batch_size = 32
train_dataset = datasets.CIFAR10(root="dataset/", train=True, 
                                 transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, 
                          shuffle=True)

num_classes = 10
learning_rate = 1e-3

Files already downloaded and verified


## Model

We are using a pretrained model of VGG16 architecture

In [4]:
# ==============================================================================
# Load VGG16
# ==============================================================================
model = torchvision.models.vgg16(pretrained=True)

# ==============================================================================
# Because we will just finetune the parameters we turn off training of them, for
# now.
# ==============================================================================
for param in model.parameters():
    param.requires_grad = False

In [5]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Accessing particular layers

Pytorch proovides multiple ways to access particular layers in your model. For example, if we want to allow training of the last conv layer in the features, we can use a subscript to access the layer through the sequential model as below

In [6]:
# ==============================================================================
# Accessing the last conv layer whose index is 28
# ==============================================================================
model.features[28]

Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [7]:
# ==============================================================================
# Assuming, we will allow the finetuning of that layer
# ==============================================================================
for param in model.features[28].parameters():
    # --------------------------------------------------------------------------
    # change to True to allow training that layer
    # --------------------------------------------------------------------------
    param.requires_grad = False

### Model Adjustment

We do not necessarily need to use the same model again. In fact, we might have some changes. Below, we change the classifier layers as below

In [8]:
# ==============================================================================
# We do not want an avgpool, so we create a custome layer that does nothing
# ==============================================================================
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [9]:
# ==============================================================================
# Customize some layers
# ==============================================================================
model.avgpool = Identity()
model.classifier = nn.Sequential(
    nn.Linear(512, 100),
    nn.ReLU(),
    nn.Linear(100, num_classes)
)

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Loss and optimizer

We use Adam as our optimizer along with CrossEntropy

In [10]:
# ==============================================================================
# Define our loss and optimizer
# ==============================================================================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [11]:
# ==============================================================================
# Just few epochs for the sake of time we have, but you can increase them
# ==============================================================================
num_epochs = 3

In [12]:
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to device
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")

Cost at epoch 0 is 1.22653
Cost at epoch 1 is 1.08582
Cost at epoch 2 is 1.02695


In [13]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


check_accuracy(train_loader, model)

Checking accuracy on training data
Got 33099 / 50000 with accuracy 66.20
